In [1]:
import geopandas as gpd
import networkx as nx
import fiona
import tqdm
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import re
import time

from shapely.ops import nearest_points
from tqdm import tqdm
from shapely.ops import cascaded_union
from shapely.wkt import loads 
from shapely.geometry import Point
from scipy.spatial import cKDTree

In [2]:
network_list = glob.glob(r'C:\Users\kurokiso\Desktop\道路ネットワーク\解析データ\notoverlap\point_data\*')
density = gpd.read_file(r'C:\Users\kurokiso\Desktop\toudai\research\analytics\土地情報\人口密度\density.gpkg')

In [3]:
def ckdnearest(gdA, gdB):
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdf = pd.concat(
        [gdA, gdB.loc[idx, gdB.columns != 'geometry'].reset_index(),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

In [4]:
rowid_list = []
density_list = []
for i in tqdm(network_list):
    # id
    rowid = i[-9:]
    rowid = re.sub(r"\D", "", rowid)
    # 座標変換
    network = gpd.read_file(i)
    network.crs = {'init': 'epsg:4326'}
    
    nearest_pop = ckdnearest(network, density)
    mean_density = nearest_pop['pop_density'].mean()
    rowid_list.append(rowid)
    density_list.append(mean_density)
    print(rowid, mean_density)
    
df = pd.DataFrame({'Rowid':rowid_list, 'density': density_list})
df.to_csv('density.csv')
#     gpd.GeoDataFrame.to_file(geo_df, "buffer_100.gpkg", driver="GPKG", encoding='utf-8')

  0%|                                                                                          | 0/206 [00:00<?, ?it/s]C:\Users\kurokiso\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
  0%|                                                                                          | 0/206 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [6]:
df

,Rowid,density
0,10,1878.298720
1,100,27924.866406
2,101,44219.445652
3,102,63127.655816
4,103,44025.092448
...,...,...
201,95,436.597056
202,96,7106.761328
203,97,27127.097656
204,98,29979.161719


In [7]:
df.to_csv("density.csv")